In [1]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv("C:\\Users\\Saffa\\Downloads\\ames\\AmesHousing.csv", na_values = [''])

# Quick look at the data
print(df.info())
print(df.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            2930 non-null   int64  
 1   PID              2930 non-null   int64  
 2   MS SubClass      2930 non-null   int64  
 3   MS Zoning        2930 non-null   object 
 4   Lot Frontage     2440 non-null   float64
 5   Lot Area         2930 non-null   int64  
 6   Street           2930 non-null   object 
 7   Alley            198 non-null    object 
 8   Lot Shape        2930 non-null   object 
 9   Land Contour     2930 non-null   object 
 10  Utilities        2930 non-null   object 
 11  Lot Config       2930 non-null   object 
 12  Land Slope       2930 non-null   object 
 13  Neighborhood     2930 non-null   object 
 14  Condition 1      2930 non-null   object 
 15  Condition 2      2930 non-null   object 
 16  Bldg Type        2930 non-null   object 
 17  House Style   

In [2]:
df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [3]:
# Creating a derived variable

df['Price_Category'] = pd.cut(df['SalePrice'], bins=[0, 150000, 300000, 600000], labels=['Low', 'Medium', 'High'])

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

categorical_cols = df.select_dtypes(include=['object']).columns

# Apply label encoding to each categorical column
for col in categorical_cols:
    df[col] = label_encoder.fit_transform(df[col])
df['Price_Category'] = label_encoder.fit_transform(df['Price_Category'])

# impute rows with missing values
from sklearn.impute import KNNImputer

# KNN Imputer
knn_imputer = KNNImputer(n_neighbors=5)
df = pd.DataFrame(knn_imputer.fit_transform(df), columns=df.columns)




In [4]:
df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice,Price_Category
0,1.0,526301100.0,20.0,5.0,141.0,31770.0,1.0,2.0,0.0,3.0,...,4.0,4.0,5.0,0.0,5.0,2010.0,9.0,4.0,215000.0,2.0
1,2.0,526350040.0,20.0,4.0,80.0,11622.0,1.0,2.0,3.0,3.0,...,4.0,2.0,5.0,0.0,6.0,2010.0,9.0,4.0,105000.0,1.0
2,3.0,526351010.0,20.0,5.0,81.0,14267.0,1.0,2.0,0.0,3.0,...,4.0,4.0,1.0,12500.0,6.0,2010.0,9.0,4.0,172000.0,2.0
3,4.0,526353030.0,20.0,5.0,93.0,11160.0,1.0,2.0,3.0,3.0,...,4.0,4.0,5.0,0.0,4.0,2010.0,9.0,4.0,244000.0,2.0
4,5.0,527105010.0,60.0,5.0,74.0,13830.0,1.0,2.0,0.0,3.0,...,4.0,2.0,5.0,0.0,3.0,2010.0,9.0,4.0,189900.0,2.0


In [5]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Price_Category'])  
y = df['Price_Category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler

# Scaling features for KNN
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Training KNN
knn_model = KNeighborsClassifier(n_neighbors=5)  
knn_model.fit(X_train_scaled, y_train)

# Training Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)


GaussianNB()

In [7]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# KNN Predictions and Evaluation
knn_predictions = knn_model.predict(X_test_scaled)
print("KNN Evaluation:")
print("Accuracy:", accuracy_score(y_test, knn_predictions))
print("Classification Report:\n", classification_report(y_test, knn_predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, knn_predictions))

# Naive Bayes Predictions and Evaluation
nb_predictions = nb_model.predict(X_test)
print("\nNaive Bayes Evaluation:")
print("Accuracy:", accuracy_score(y_test, nb_predictions))
print("Classification Report:\n", classification_report(y_test, nb_predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, nb_predictions))


KNN Evaluation:
Accuracy: 0.8481228668941979
Classification Report:
               precision    recall  f1-score   support

         0.0       0.73      0.49      0.59        49
         1.0       0.88      0.94      0.91       286
         2.0       0.83      0.82      0.82       250
         3.0       0.00      0.00      0.00         1

    accuracy                           0.85       586
   macro avg       0.61      0.56      0.58       586
weighted avg       0.84      0.85      0.84       586

Confusion Matrix:
 [[ 24   0  25   0]
 [  0 268  18   0]
 [  8  37 205   0]
 [  1   0   0   0]]

Naive Bayes Evaluation:
Accuracy: 0.9402730375426621
Classification Report:
               precision    recall  f1-score   support

         0.0       0.94      0.92      0.93        49
         1.0       0.91      1.00      0.95       286
         2.0       0.99      0.88      0.93       250
         3.0       0.50      1.00      0.67         1

    accuracy                           0.94       

C:\Users\Saffa\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Saffa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Saffa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-d

# KNN Evaluation

# Naive Bayes Evaluation

# Key Observations

In [8]:
#to ignore warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

#KNN Optimization

from sklearn.model_selection import GridSearchCV

# Define a range for 'n_neighbors'
knn_params = {'n_neighbors': list(range(1, 31))}

# Grid search with 5-fold cross-validation
grid_knn = GridSearchCV(KNeighborsClassifier(), knn_params, cv=5, scoring='accuracy')
grid_knn.fit(X_train_scaled, y_train)

# Best parameters and best score
best_knn_model = grid_knn.best_estimator_
print("Best KNN Parameters:", grid_knn.best_params_)
print("Best KNN Accuracy:", grid_knn.best_score_)

# Evaluate the best KNN model on the test set
knn_test_predictions = best_knn_model.predict(X_test_scaled)
print("\nOptimized KNN Test Set Performance:")
print("Accuracy:", accuracy_score(y_test, knn_test_predictions))
print("Classification Report:\n", classification_report(y_test, knn_test_predictions))


Best KNN Parameters: {'n_neighbors': 9}
Best KNN Accuracy: 0.8634820403477121

Optimized KNN Test Set Performance:
Accuracy: 0.8600682593856656
Classification Report:
               precision    recall  f1-score   support

         0.0       0.86      0.51      0.64        49
         1.0       0.87      0.95      0.91       286
         2.0       0.84      0.83      0.84       250
         3.0       0.00      0.00      0.00         1

    accuracy                           0.86       586
   macro avg       0.64      0.57      0.60       586
weighted avg       0.86      0.86      0.85       586



C:\Users\Saffa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Saffa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Saffa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
#Naive Bayes Optimization

from sklearn.naive_bayes import GaussianNB

# Define a range for 'var_smoothing'
nb_params = {'var_smoothing': np.logspace(-9, 0, 10)}

# Grid search with 5-fold cross-validation
grid_nb = GridSearchCV(GaussianNB(), nb_params, cv=5, scoring='accuracy')
grid_nb.fit(X_train, y_train)

# Best parameters and best score
best_nb_model = grid_nb.best_estimator_
print("Best Naive Bayes Parameters:", grid_nb.best_params_)
print("Best Naive Bayes Accuracy:", grid_nb.best_score_)

# Evaluate the best Naive Bayes model on the test set
nb_test_predictions = best_nb_model.predict(X_test)
print("\nOptimized Naive Bayes Test Set Performance:")
print("Accuracy:", accuracy_score(y_test, nb_test_predictions))
print("Classification Report:\n", classification_report(y_test, nb_test_predictions))


Best Naive Bayes Parameters: {'var_smoothing': 1e-08}
Best Naive Bayes Accuracy: 0.950080185154812

Optimized Naive Bayes Test Set Performance:
Accuracy: 0.9436860068259386
Classification Report:
               precision    recall  f1-score   support

         0.0       0.98      0.90      0.94        49
         1.0       0.91      1.00      0.95       286
         2.0       0.99      0.89      0.93       250
         3.0       0.33      1.00      0.50         1

    accuracy                           0.94       586
   macro avg       0.80      0.95      0.83       586
weighted avg       0.95      0.94      0.94       586



# KNN (Optimized)

# Naive Bayes (Optimized)

# Key Observations:

# Conclusion: